In [1]:
import os
import json
import pandas as pd
import numpy as np

folder_path = r'C:\Users\Shu\Desktop\Bike sharing\Dataset' 
all_trips = []

for day_idx, fname in enumerate(sorted(os.listdir(folder_path))):
    if fname.endswith('.json') and fname.startswith('simu0'):
        with open(os.path.join(folder_path, fname)) as f:
            trips = json.load(f)
            for trip in trips:
                all_trips.append({
                    'day': day_idx-1,
                    'time_departure': trip[0],
                    'station_departure': trip[1],
                    'time_arrival': trip[2],
                    'station_arrival': trip[3]
                })

df = pd.DataFrame(all_trips)
df


,day,time_departure,station_departure,time_arrival,station_arrival
0,1,4.0,1.0,22.0,11.0
1,1,15.0,1.0,37.0,18.0
2,1,19.0,12.0,45.0,8.0
3,1,48.0,11.0,72.0,5.0
4,1,50.0,22.0,73.0,27.0
...,...,...,...,...,...
732534,500,1431.0,16.0,15.0,10.0
732535,500,1434.0,15.0,18.0,14.0
732536,500,1437.0,8.0,2.0,0.0
732537,500,1437.0,8.0,24.0,1.0


In [2]:
from itertools import product


departures = (
    df[['day','time_departure','station_departure']]
    .rename(columns={'time_departure':'time','station_departure':'station'})
)
departures['departure_count'] = 1

arrivals = (
    df[['day','time_arrival','station_arrival']]
    .rename(columns={'time_arrival':'time','station_arrival':'station'})
)
arrivals['arrival_count'] = 1

events = pd.concat([departures, arrivals], ignore_index=True)
events[['departure_count','arrival_count']] = events[['departure_count','arrival_count']].fillna(0)
events['interval_30min'] = (events['time'] // 30).astype(int)

agg = (
    events
    .groupby(['day','interval_30min','station'])[['departure_count','arrival_count']]
    .sum()
    .reset_index()
)
agg['net_growth'] = agg['arrival_count'] - agg['departure_count']

all_days      = df['day'].unique()
all_intervals = list(range(48))
all_stations  = sorted(pd.unique(
    pd.concat([
        df['station_departure'],
        df['station_arrival']
    ])
))

idx = pd.MultiIndex.from_product(
    [all_days, all_intervals, all_stations],
    names=['day','interval_30min','station']
)

df_demand = (
    agg
    .set_index(['day','interval_30min','station'])
    .reindex(idx, fill_value=0)
    .reset_index()
)


print(df_demand.shape)  
df_demand


(720000, 6)


,day,interval_30min,station,departure_count,arrival_count,net_growth
0,1,0,0.0,0.0,0.0,0.0
1,1,0,1.0,2.0,0.0,-2.0
2,1,0,2.0,0.0,0.0,0.0
3,1,0,3.0,0.0,0.0,0.0
4,1,0,4.0,0.0,0.0,0.0
...,...,...,...,...,...,...
719995,500,47,25.0,0.0,0.0,0.0
719996,500,47,26.0,1.0,1.0,0.0
719997,500,47,27.0,1.0,0.0,-1.0
719998,500,47,28.0,0.0,0.0,0.0


In [11]:
df_demand.to_csv("raw_data.csv", index=False)

##  Bike sharing system

In [3]:
# Load two more data
with open(r"C:\Users\Shu\Desktop\Bike sharing\Dataset\Initial_Inven.json") as f:
    initial_inventory = json.load(f)

# Load distance matrix
with open(r"C:\Users\Shu\Desktop\Bike sharing\Dataset\Dis.json") as f:
    dist_matrix = json.load(f)

# Convert to numpy array for ease
dist_matrix = np.array(dist_matrix)
transit_time = 2 * dist_matrix  # α = 2

In [5]:
# Define variables
S = list(range(30))   # Stations
V = list(range(2))    # Vehicles
T = list(range(48))   # 48 half-hour periods (full day)

# Parameters
station_capacity = {s: 40 if s < 5 else 20 for s in S}
vehicle_capacity = 40
initial_vehicle_inventory = [20, 20]


In [69]:
df_day1 = df_demand[df_demand["day"] == 1]

f_plus = {(s, t): 0 for s in S for t in T}
f_minus = {(s, t): 0 for s in S for t in T}

for _, row in df_day1.iterrows():
    s = int(row["station"])
    t = int(row["interval_30min"])
    f_plus[(s, t)] = row["departure_count"]
    f_minus[(s, t)] = row["arrival_count"]


In [76]:
m = Model("Bike Rebalancing Day 1")

r_plus = m.addVars(S, T, V, name="r_plus", vtype=GRB.INTEGER)
r_minus = m.addVars(S, T, V, name="r_minus", vtype=GRB.INTEGER)
z = m.addVars(S, T, V, name="z", vtype=GRB.BINARY)
d = m.addVars(S, T, name="d", vtype=GRB.INTEGER)
d_hat = m.addVars(V, T, name="d_hat", vtype=GRB.INTEGER)
x_plus = m.addVars(S, T, name="x_plus", vtype=GRB.INTEGER)
x_minus = m.addVars(S, T, name="x_minus", vtype=GRB.INTEGER)

m.setObjective(
    quicksum(f_plus[s, t] - x_plus[s, t] for s in S for t in T) +
    quicksum(f_minus[s, t] - x_minus[s, t] for s in S for t in T),
    GRB.MINIMIZE
)

for t in T[:-1]:
    for v in V:
        m.addConstr(d_hat[v, t + 1] == d_hat[v, t] +
                    quicksum(r_plus[s, t, v] - r_minus[s, t, v] for s in S))

for t in T[:-1]:
    for s in S:
        m.addConstr(d[s, t + 1] == d[s, t] -
                    quicksum(r_plus[s, t, v] - r_minus[s, t, v] for v in V) -
                    x_plus[s, t] + x_minus[s, t])

for t in T:
    for v in V:
        m.addConstr(quicksum(z[s, t, v] for s in S) == 1)

for t in T:
    for s in S:
        for v in V:
            m.addConstr(r_plus[s, t, v] + r_minus[s, t, v] <= vehicle_capacity * z[s, t, v])

for t in T:
    for s in S:
        m.addConstr(d[s, t] <= station_capacity[s])
    for v in V:
        m.addConstr(d_hat[v, t] <= vehicle_capacity)

for s in S:
    for t in T:
        m.addConstr(x_plus[s, t] <= f_plus[s, t])
        m.addConstr(x_minus[s, t] <= f_minus[s, t])

for s in S:
    for t in T:
        for v in V:
            m.addConstr(r_plus[s, t, v] <= vehicle_capacity)
            m.addConstr(r_minus[s, t, v] <= vehicle_capacity)

for s in S:
    m.addConstr(d[s, 0] == int(initial_inventory[s]))
for v in V:
    m.addConstr(d_hat[v, 0] == initial_vehicle_inventory[v])

m.optimize()

if m.status == GRB.OPTIMAL:
    print("✅ Optimal total unsatisfied demand:", m.objVal)
    truck_actions = []
    for v in V:
        for t in T:
            for s in S:
                if z[s, t, v].X > 0.5:
                    action_record = {
                        "vehicle": v,
                        "time": t,
                        "station": s,
                        "pickup": int(r_plus[s, t, v].X),
                        "dropoff": int(r_minus[s, t, v].X)
                    }
                    truck_actions.append(action_record)
    
    df_actions = pd.DataFrame(truck_actions)
    print(df_actions.head())
else:
    print("❌ Optimization ended with status:", m.status)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-10210U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 14688 rows, 13056 columns and 38836 nonzeros
Model fingerprint: 0x9fcbc775
Variable types: 0 continuous, 13056 integer (2880 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+01]
Presolve removed 10364 rows and 1772 columns
Presolve time: 0.06s
Presolved: 4324 rows, 11284 columns, 26762 nonzeros
Variable types: 0 continuous, 11284 integer (3409 binary)

Root relaxation: objective 0.000000e+00, 4533 iterations, 0.21 seconds (0.25 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.000

In [12]:
from gurobipy import Model, GRB, quicksum


S = list(range(30))    # stations 0–29
T = list(range(48))    # 48 half‑hour intervals per day
V = list(range(2))     # two vehicles

station_capacity = {s: 40 if s < 5 else 20 for s in S}
vehicle_capacity = 40

# -----------------------------------------------------------------------------
# 3) Initial inventories for the very first day (day 400)
# -----------------------------------------------------------------------------
initial_inventory = [
    7, 0, 1, 9, 0, 9, 5, 18, 3, 5,
    20, 20, 20, 10, 5, 5, 8, 19, 9, 6,
    13, 20, 14, 17, 6, 14, 6, 15, 13, 7
]
initial_vehicle_inventory = [20, 20]

# -----------------------------------------------------------------------------
# 4) Precompute each day’s true demand profile (days 400–500)
# -----------------------------------------------------------------------------
f_plus_dict = {}   # departures
f_minus_dict = {}  # arrivals

for day in range(400, 501): 
    daily = (
        df_demand[df_demand["day"] == day]
        .groupby(["station", "interval_30min"])
        [["departure_count", "arrival_count"]]
        .sum()
        .reset_index()
    )

    # start from zeros
    f_plus = {(s, t): 0 for s in S for t in T}
    f_minus = {(s, t): 0 for s in S for t in T}

    # fill in actual counts
    for _, row in daily.iterrows():
        s = int(row["station"])
        t = int(row["interval_30min"])
        f_plus[(s, t)]  = int(row["departure_count"])
        f_minus[(s, t)] = int(row["arrival_count"])

    f_plus_dict[day]  = f_plus
    f_minus_dict[day] = f_minus

In [13]:
baseline_objs = []  # to store (day, objective value)

for day in range(400, 501):
    fp = f_plus_dict[day]
    fm = f_minus_dict[day]

    m = Model(f"BikeRebalance_Day{day}")
    m.setParam("OutputFlag", 0)

    # variables
    r_plus  = m.addVars(S, T, V, vtype=GRB.INTEGER, name="r_plus")
    r_minus = m.addVars(S, T, V, vtype=GRB.INTEGER, name="r_minus")
    z       = m.addVars(S, T, V, vtype=GRB.BINARY,  name="z")
    d       = m.addVars(S, T,     vtype=GRB.INTEGER, name="d")
    d_hat   = m.addVars(V, T,     vtype=GRB.INTEGER, name="d_hat")
    x_plus  = m.addVars(S, T,     vtype=GRB.INTEGER, name="x_plus")
    x_minus = m.addVars(S, T,     vtype=GRB.INTEGER, name="x_minus")

    # objective
    obj = quicksum(fp[s, t] - x_plus[s, t] for s in S for t in T) + \
          quicksum(fm[s, t] - x_minus[s, t] for s in S for t in T)
    m.setObjective(obj, GRB.MINIMIZE)

    # constraints (same as before)
    for t in T[:-1]:
        for v in V:
            m.addConstr(d_hat[v, t + 1] == d_hat[v, t] +
                        quicksum(r_plus[s, t, v] - r_minus[s, t, v] for s in S))
    for t in T[:-1]:
        for s in S:
            m.addConstr(d[s, t + 1] == d[s, t] -
                        quicksum(r_plus[s, t, v] - r_minus[s, t, v] for v in V) -
                        x_plus[s, t] + x_minus[s, t])
    for t in T:
        for v in V:
            m.addConstr(quicksum(z[s, t, v] for s in S) == 1)
        for s in S:
            for v in V:
                m.addConstr(r_plus[s, t, v] + r_minus[s, t, v] <= vehicle_capacity * z[s, t, v])
        for s in S:
            m.addConstr(d[s, t] <= station_capacity[s])
        for v in V:
            m.addConstr(d_hat[v, t] <= vehicle_capacity)
    for s in S:
        for t in T:
            m.addConstr(x_plus[s, t] <= fp[(s, t)])
            m.addConstr(x_minus[s, t] <= fm[(s, t)])
            for v in V:
                m.addConstr(r_plus[s, t, v] <= vehicle_capacity)
                m.addConstr(r_minus[s, t, v] <= vehicle_capacity)
    for s in S:
        m.addConstr(d[s, 0] == inv[s])
    for v in V:
        m.addConstr(d_hat[v, 0] == v_inv[v])

    # solve
    m.optimize()
    if m.status == GRB.OPTIMAL:
        obj_val = m.ObjVal
        print(f"✅ Day {day} – Optimal unmet demand: {obj_val}")
        baseline_objs.append((day, obj_val))
        # update for next day
        last_t = T[-1]
        inv = [int(d[s, last_t].X) for s in S]
        v_inv = [int(d_hat[v, last_t].X) for v in V]
    else:
        print(f"❌ Day {day} – Solver failed with status {m.status}")
        baseline_objs.append((day, None))


✅ Day 400 – Optimal unmet demand: 0.0
✅ Day 401 – Optimal unmet demand: 0.0
✅ Day 402 – Optimal unmet demand: 0.0
✅ Day 403 – Optimal unmet demand: 2.0
✅ Day 404 – Optimal unmet demand: 0.0
✅ Day 405 – Optimal unmet demand: 1.0
✅ Day 406 – Optimal unmet demand: 1.0
✅ Day 407 – Optimal unmet demand: 0.0
✅ Day 408 – Optimal unmet demand: 0.0
✅ Day 409 – Optimal unmet demand: 0.0
✅ Day 410 – Optimal unmet demand: 0.0
✅ Day 411 – Optimal unmet demand: 1.0
✅ Day 412 – Optimal unmet demand: 0.0
✅ Day 413 – Optimal unmet demand: 0.0
✅ Day 414 – Optimal unmet demand: 0.0
✅ Day 415 – Optimal unmet demand: 0.0
✅ Day 416 – Optimal unmet demand: 0.0
✅ Day 417 – Optimal unmet demand: 0.0
✅ Day 418 – Optimal unmet demand: 1.0
✅ Day 419 – Optimal unmet demand: 0.0
✅ Day 420 – Optimal unmet demand: 0.0
✅ Day 421 – Optimal unmet demand: 0.0
✅ Day 422 – Optimal unmet demand: 0.0
✅ Day 423 – Optimal unmet demand: 1.0
✅ Day 424 – Optimal unmet demand: 0.0
✅ Day 425 – Optimal unmet demand: 1.0
✅ Day 426 – 

In [14]:
df_obj = pd.DataFrame(baseline_objs, columns=["day", "unmet_demand"])
average_lost_demand = df_obj["unmet_demand"].mean()
print(f"📉 Average lost (unmet) demand: {average_lost_demand:.2f}")


📉 Average lost (unmet) demand: 0.37


In [4]:
# ML f plus and f minus
df_ml = pd.read_csv(r"C:\Users\Shu\Downloads\Raw_data.csv")

In [9]:
from gurobipy import Model, GRB, quicksum


S = list(range(30))    # stations 0–29
T = list(range(48))    # 48 half‑hour intervals per day
V = list(range(2))     # two vehicles

station_capacity = {s: 40 if s < 5 else 20 for s in S}
vehicle_capacity = 40

# -----------------------------------------------------------------------------
# 3) Initial inventories for the very first day (day 400)
# -----------------------------------------------------------------------------
inv = [
    7, 0, 1, 9, 0, 9, 5, 18, 3, 5,
    20, 20, 20, 10, 5, 5, 8, 19, 9, 6,
    13, 20, 14, 17, 6, 14, 6, 15, 13, 7
]
v_inv= [20, 20]

# -----------------------------------------------------------------------------
# 4) Precompute each day’s true demand profile (days 400–500)
# -----------------------------------------------------------------------------
f_plus_dict = {}   # departures
f_minus_dict = {}  # arrivals

for day in range(400, 501): 
    daily = (
        df_ml[df_ml["day"] == day]
        .groupby(["station", "interval_30min"])
        [["departure_count", "arrival_count"]]
        .sum()
        .reset_index()
    )

    # start from zeros
    f_plus = {(s, t): 0 for s in S for t in T}
    f_minus = {(s, t): 0 for s in S for t in T}

    # fill in actual counts
    for _, row in daily.iterrows():
        s = int(row["station"])
        t = int(row["interval_30min"])
        f_plus[(s, t)]  = int(row["departure_count"])
        f_minus[(s, t)] = int(row["arrival_count"])

    f_plus_dict[day]  = f_plus
    f_minus_dict[day] = f_minus

In [10]:
baseline_objs = []  # to store (day, objective value)

for day in range(400, 501):
    fp = f_plus_dict[day]
    fm = f_minus_dict[day]

    m = Model(f"BikeRebalance_Day{day}")
    m.setParam("OutputFlag", 0)
    m.Params.TimeLimit = 5

    # variables
    r_plus  = m.addVars(S, T, V, vtype=GRB.INTEGER, name="r_plus")
    r_minus = m.addVars(S, T, V, vtype=GRB.INTEGER, name="r_minus")
    z       = m.addVars(S, T, V, vtype=GRB.BINARY,  name="z")
    d       = m.addVars(S, T,     vtype=GRB.INTEGER, name="d")
    d_hat   = m.addVars(V, T,     vtype=GRB.INTEGER, name="d_hat")
    x_plus  = m.addVars(S, T,     vtype=GRB.INTEGER, name="x_plus")
    x_minus = m.addVars(S, T,     vtype=GRB.INTEGER, name="x_minus")

    # objective
    obj = quicksum(fp[s, t] - x_plus[s, t] for s in S for t in T) + \
          quicksum(fm[s, t] - x_minus[s, t] for s in S for t in T)
    m.setObjective(obj, GRB.MINIMIZE)

    # constraints (same as before)
    for t in T[:-1]:
        for v in V:
            m.addConstr(d_hat[v, t + 1] == d_hat[v, t] +
                        quicksum(r_plus[s, t, v] - r_minus[s, t, v] for s in S))
    for t in T[:-1]:
        for s in S:
            m.addConstr(d[s, t + 1] == d[s, t] -
                        quicksum(r_plus[s, t, v] - r_minus[s, t, v] for v in V) -
                        x_plus[s, t] + x_minus[s, t])
    for t in T:
        for v in V:
            m.addConstr(quicksum(z[s, t, v] for s in S) == 1)
        for s in S:
            for v in V:
                m.addConstr(r_plus[s, t, v] + r_minus[s, t, v] <= vehicle_capacity * z[s, t, v])
        for s in S:
            m.addConstr(d[s, t] <= station_capacity[s])
        for v in V:
            m.addConstr(d_hat[v, t] <= vehicle_capacity)
    for s in S:
        for t in T:
            m.addConstr(x_plus[s, t] <= fp[(s, t)])
            m.addConstr(x_minus[s, t] <= fm[(s, t)])
            for v in V:
                m.addConstr(r_plus[s, t, v] <= vehicle_capacity)
                m.addConstr(r_minus[s, t, v] <= vehicle_capacity)
    for s in S:
        m.addConstr(d[s, 0] == inv[s])
    for v in V:
        m.addConstr(d_hat[v, 0] == v_inv[v])
    m.setParam("Seed", 42)

    # solve
    m.optimize()
    if m.status == GRB.OPTIMAL:
        obj_val = m.ObjVal
        print(f"✅ Day {day} – Optimal unmet demand: {obj_val}")
        baseline_objs.append((day, obj_val))
        # update for next day
        last_t = T[-1]
        inv = [int(d[s, last_t].X) for s in S]
        v_inv = [int(d_hat[v, last_t].X) for v in V]
    else:
        print(f"❌ Day {day} – Solver failed with status {m.status}")
        baseline_objs.append((day, None))


✅ Day 400 – Optimal unmet demand: 0.0
✅ Day 401 – Optimal unmet demand: 0.0
✅ Day 402 – Optimal unmet demand: 0.0
✅ Day 403 – Optimal unmet demand: 3.0
✅ Day 404 – Optimal unmet demand: 0.0
✅ Day 405 – Optimal unmet demand: 0.0
✅ Day 406 – Optimal unmet demand: 0.0
✅ Day 407 – Optimal unmet demand: 0.0
✅ Day 408 – Optimal unmet demand: 0.0
✅ Day 409 – Optimal unmet demand: 0.0
✅ Day 410 – Optimal unmet demand: 0.0
✅ Day 411 – Optimal unmet demand: 0.0
✅ Day 412 – Optimal unmet demand: 0.0
✅ Day 413 – Optimal unmet demand: 0.0
✅ Day 414 – Optimal unmet demand: 0.0
✅ Day 415 – Optimal unmet demand: 0.0
✅ Day 416 – Optimal unmet demand: 0.0
✅ Day 417 – Optimal unmet demand: 0.0
✅ Day 418 – Optimal unmet demand: 0.0
✅ Day 419 – Optimal unmet demand: 0.0
✅ Day 420 – Optimal unmet demand: 0.0
✅ Day 421 – Optimal unmet demand: 2.0
✅ Day 422 – Optimal unmet demand: 1.0
✅ Day 423 – Optimal unmet demand: 1.0
✅ Day 424 – Optimal unmet demand: 0.0
✅ Day 425 – Optimal unmet demand: 1.0
✅ Day 426 – 

In [11]:
df_obj_ml = pd.DataFrame(baseline_objs, columns=["day", "unmet_demand"])
average_lost_demand_ml = df_obj_ml["unmet_demand"].mean()
print(f"📉 Average lost (unmet) demand: {average_lost_demand_ml:.2f}")


📉 Average lost (unmet) demand: 0.31
